# Visualize json
## MSBX 5420: Week 7
This notebook parses Call of Duty World League match data and plots the spawns and deaths of every player across multiple tournaments.

In [1]:
import os
import json
import ujson
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FCagg
from matplotlib.figure import Figure
import glob

In [10]:
# function to call the map image, plot the spawns/deaths, and export the new image
def visualize(data, background, image):
    death_pos, spawn_pos = {'x': [], 'y': []}, {'x': [], 'y': []}
    # get pos data
    events = data['events']
    for evt in events:
        if evt['type'] == 'spawn':
            spawn_pos['x'].append( evt['data']['pos']['x'] )
            spawn_pos['y'].append( evt['data']['pos']['y'] )
        if evt['type'] == 'death':
            death_pos['x'].append( evt['data']['pos']['x'] )
            death_pos['y'].append( evt['data']['pos']['y'] )
            
    # create figure
    fig = Figure()
    FCagg(fig)
    ax = fig.subplots(1,1)
    
    fig.set_figheight(15)
    fig.set_figwidth(15)
    
    # plot map
    map_image = plt.imread(background)
    ax.imshow(map_image)
    ax.set_facecolor((210/255.0, 210/255.0, 210/255.0))
    
    # plot spawns
    ax.scatter(spawn_pos['x'], spawn_pos['y'], color='blue')
    # plot death events
    ax.scatter(death_pos['x'], death_pos['y'], color='red')
    # set title
    ax.set_title('{} ({})'.format(data['map'], data['mode']))
    
    fig.savefig(image)


In [17]:
# setting paths for data and maps
datadirs = glob.glob('structured-2018-*')

mapdir = os.path.join('..', 'week7', 'maps', 'ww2')

# dictionary reference for each map name and its image file
map_images = {
    'Aachen': 'aachen.png',
    'Ardennes Forest': 'ardennes_forest.png',
    'Flak Tower': 'flaktower.png',
    'Gibraltar': 'gibraltar.png',
    'London Docks': 'london_docks.png',
    'Sainte Marie du Mont': 'sainte_marie_du_mont.png' ,
    'USS Texas': 'uss_texas.png',
    'Valkyrie': 'valkyrie.png'
}

In [18]:
# function to parse each json and call the appropriate map
def parsejson(file, image):
    with open(file, 'r') as f:
        rawdata = f.read()
    data = ujson.loads(rawdata)
    
    mapname = data['map']
    if mapname in map_images:
        mapfile = map_images[mapname]
        background = os.path.join(mapdir, mapfile)
        visualize(data, background, image)

In [20]:
# loop through directories and json files
for datadir in datadirs:
    imagedir = 'maps_' + datadir
    if not os.path.exists(imagedir):
        os.mkdir(imagedir)
    files = glob.glob(os.path.join(datadir, '*.json'))
    
    for file in files:
        basename = os.path.basename(file)
        no_ext = os.path.splitext(basename)[0]
        png = no_ext + '.png'
        #name = file[:-5] + '.png'
        image = os.path.join(imagedir, png)
        parsejson(file, image)